In [1]:
# Modelling and Analysis of Data
# Exam 2022 : Date  17th - 25th of January
# Exam no: 39

# Question 7 (K-means Clustering & Principal Component Analysis, 7 points)
# Question A and B

import numpy as np

In [2]:
def normalizeData(x):
    '''
    Normalizes the input using mean and the standard deviation.
    Normalization is done on all the columns in the 2d numpy-array 'x'

    Returns the normalized 'x'
    '''
    return (x - np.mean(x, axis=0)) / np.std(x, axis=0)



def euclideanDistance(a, b):
    '''
    Computes the distance from a to b

    Returns the distance
    '''
    return np.linalg.norm(a-b)


def generate_centroids(k, data, randomGeneratedNumbers):
    """
    Randomly choose k datapoints from data matrix to be used as seed centroids

    Parameters:
    -----------
    k                       : number of centroids
    data                    : (n_samples, n_features) datamatrix
    randomGeneratedNumbers  : np random generator

    Returns:
    --------
    centroids : Numpy array of k random centroids
    """
    centroids = np.empty((0,data.shape[1]))

    for _ in range(k):
        i = int(randomGeneratedNumbers.random() * data.shape[0])
        datapoint = data[i]
        centroids = np.vstack((centroids, datapoint))

    return centroids



def find_nearest_centroid(datapoint, centroids):
    """
    Computes the index of the nearest centroid

    Params:
    ------
    datapoint : (n_features) np.array
    centroid  : (k, n_features) np.array of centroids

    Returns:
    -------
    index of neares centroid

    """
    distances = [euclideanDistance(datapoint, x) for x in centroids]
    return np.argsort(distances)[0]



def assign_datapoints_to_centroids(data, centroids):
    """
    Assign datapoints to nearest centroids

    Params:
    -------
    data      : (n_samples, n_features) data matrix
    centroids : (k, n_features) array of centroids

    Returns:
    --------
    assignments : np.array of indices mapping each datapoint to
    its nearest centroid
    """
    assignments = [find_nearest_centroid(datapoint, centroids) for datapoint in data]
    return np.array(assignments)



def compute_sum_intra_cluster_dist(data, assignments, centroids):
    """
    Compute the sum of intra cluster distances of all k clusters

    Params:
    -------
    data          : (n_samples, n_features) data matrix
    assignments   : (n_samples) np.array
    centroids     : (k, n_features)


    Returns:
    -------
    computed intra cluster distance
    """
    k = len(centroids)

    # 3D-array of k clusters with assigned datapoint
    clusters = np.array([data[np.where(assignments == j)] for j in range(k)], dtype=object)

    sum = 0

    for j in range(k):
        sum += calculate_intra_cluster_dist(clusters[j], centroids[j])

    return sum



def calculate_intra_cluster_dist(cluster, centroid):
    """
    Compute the intra cluster distance of a single cluster
    """
    return np.sum([euclideanDistance(datapoint, centroid) for datapoint in cluster])



def calculate_new_centroids(data, assignments, centroids):
    """
    Compute new centroids

    Params:
    -------
    data          : (n_samples, n_features) data matrix
    assignments   : (n_samples) np.array
    centroids     : (k, n_features)

    Returns:
    --------
    centroids : (k, n_features) np.array
       The new centroids
    """
    k = len(centroids)

    # 3D-array of k clusters with assigned datapoints
    clusters = np.array([data[np.where(assignments == j)] for j in range(k)], dtype=object)

    for j in range(k):
        # number of datapoints in j'th cluster
        n = clusters[j].shape[0]
        if (n > 0):
            # update j'th centroid
            centroids[j] = 1/n * np.sum(clusters[j], axis=0)

    return centroids

def k_mean_clustering(k, data, centroids):
    """
    K-means clustering

    Params:
    ------
    k : number of clusters
    data : (n_samples, n_features) (normalized) data matrix

    Returns:
    --------
    assignments, intra_cluster_dist : tuple

    """
    current_assignments = assign_datapoints_to_centroids(data, centroids)

    new_assignments = [] # initial empty value

    while not np.array_equal(current_assignments, new_assignments):
        # repeat until assignments does not change
        centroids = calculate_new_centroids(data, current_assignments, centroids)
        current_assignments = new_assignments
        new_assignments = assign_datapoints_to_centroids(data, centroids)

    intra_cluster_dist = compute_sum_intra_cluster_dist(data, current_assignments, centroids)
    
    return current_assignments, intra_cluster_dist




data = np.loadtxt('./seedsDataset.txt', delimiter=',')

print('Data shape:', data.shape)

# Number of clusters
k = 3
seed_value = 1
normalized_data = normalizeData(data)

# Constructs new generator with a specific seed
randomGeneratedNumbers = np.random.default_rng(seed_value)

# running K-means clustering 5 times
result = np.empty((0,3))

for i in range(5):
    centroids = generate_centroids(k,normalized_data, randomGeneratedNumbers)
    current_assignments, intra_cluster_dist = k_mean_clustering(k, normalized_data, centroids)
    result = np.vstack((result, [centroids, current_assignments, intra_cluster_dist]))

# Sorting to get results
centroids, current_assignments, intra_cluster_dist = result[np.argsort(result[:,2])[0]]
print("Calculations with seed value:", seed_value)
print("Smallest Intra-Cluster Distance: %.4f" %intra_cluster_dist)

# 3D-array of k clusters with assigned datapoints
clusters = np.array([normalized_data[np.where(current_assignments == j)] for j in range(k)], dtype=object)

# print number of samples in each cluster
for j in range(k):
    print("Number of samples in cluster %d: %d" %(j, len(clusters[j])))


Data shape: (200, 8)
Calculations with seed value: 1
Smallest Intra-Cluster Distance: 268.5510
Number of samples in cluster 0: 67
Number of samples in cluster 1: 66
Number of samples in cluster 2: 67


G:\Developer_Libraries\Python\lib\site-packages\numpy\core\shape_base.py:121: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ary = asanyarray(ary)
